# 2022 Systematic Trading Education Ceritificate Skeleton Code for the Coursework


## Introduction
The code included in this Notebook is the same skeleton code that we discussed in session 1 of the lecture series and the task of this coursework will be for you to write your own strategy and backtest it as we did for basic relative and time series momentum in the first lecture.

In [ ]:
pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import yfinance as yf
import matplotlib.pyplot as plt
import datetime

## The Parent Class and Evaluation function.
In the code below we define the Strategies class which is a class containing all basic features that our individual backtesting strategy classes will inherit from.

These basic features include:

- the **init method** which defines the asset tickers that the strategy will be focussed on.
- the **import data method** that will import data for a given start and end date using the yfinance module as mentioned earlier.
- the **backtest method** that defines the class variable self.strat which is the pandas data frame of values representing the strategy. This is left empty as it will be filled in in the child class.
- the **evaluate method** which calculates the cumulative returns of the given strategy, plots a chart of the cumulative returns and calculates the sharpe ratio of the strategy.
(note that one task this week will be to write code in the evaluate method so that it also returns the sortino ratio)

In [ ]:
####### THIS CODE IS THE SAME AS THE SKELETON CODE THAT WE COMPLETED IN THE FIRST LECTURE ########
####### PLEASE DON'T MESS WITH THIS CODE BLOCK AS IT WILL MAKE MARKING AN ABSOLUTE BALL-ACHE!!!!! #########
class Strategies():
  """
  A class that contains code that strategies later on will inherit from.
  params:
  -----
  codes = list of stock short codes
  -----
  """
  def __init__(self, codes):
    self.codes = codes
    self.strat = pd.DataFrame()
    self.data = pd.DataFrame()

  def import_data(self, start_date, end_date):
    """downloads all data for each backtest from yahoo Finance."""
    data = yf.download(self.codes, start_date, end_date)
    # if only one stock code is entered data is reformated so that
    # it is the same format as when multiple stocks are entered
    if len(self.codes) == 1:
      data.columns = [data.columns, self.codes*len(data.columns)]
    #returns data where any row containing NAN values is removed
    return data.dropna()

  def backtest(self, start_date, end_date):
    """
    Returns a list with elements of a time series' from yahoo finance as well as
    an array of values between -1 and 1 that represent the strategy over the giv
    en period with 1 representing a long postion in one stock, 0 representing a
    neutral postion and -1 representing a short position.
    params:
    -----
    start_date, end_date  = string of dates for backtesting with format Y-m-d
    -----
    """
    # sets up a dataframe to contain all strategy info for each stock at each 
    # time index
    self.data = self.import_data(start_date, end_date)
    self.strat = pd.DataFrame(data = np.zeros([len(self.data), len(self.codes)]),
                              columns = self.codes, index = self.data.index)
  
  def evaluate(self, start_date, end_date, fig_strat=True, fig_other=False,
               percentage_risk_free_rate = 0.1, **kwargs):
    """
    returns a dataframe with columns icluding the daily returns of the portfolio,
    the cumulative returns, the sharp ratio and all relevant plots of both the 
    stock price of each stock 
    and the strategy.
    params:
    ----
    start_date, end_date  = string of dates for backtesting with format Y-m-d
    fig = boolean variable that can be used to produce figures
    risk_free_rate = average rate of return on a very safe government issued bond
                     used to calculate the sharpe ratio with
    **kwargs are any specific keyword arguments that can be passed to the
    backtesting function to allow for comparison of the backtest for different
    possible parameters defined in the subclass.
    ----
    """
    # convert the monthly risk free rate to the daily rate for use
    # when calculating Sharpe and sortino ratios
    daily_rate = (1+ percentage_risk_free_rate/100)**(1/20) - 1
    # run the backtest function and define the stock price data to be once again
    # self.data and the signals self.strat
    strat = self.backtest(start_date, end_date, **kwargs)
    
    # sets up a new dataframe which will give the returns of the portfolio
    return_df = pd.DataFrame(columns= ["daily returns", "cumulative returns"],
                                  index = self.data.index)
    return_df["daily returns"][0] = 0
    
    # loops through the remaining dates and calculates the return across 
    # the portfolio
    for i in range(1, len(self.data)):
      return_df["daily returns"][i] = sum(100*self.strat[c][i-1]*(self.data["Adj Close"][c][i] - self.data["Adj Close"][c][i-1]) 
                                               /self.data["Adj Close"][c][i-1] for c in self.codes)
    # calculates the cumulative return for each date
    return_df["cumulative returns"] = ((return_df["daily returns"]/100+1).cumprod()-1)*100
    return_df.dropna()

    # calculates the sharpe ratio based on a low risk asset
    zero_count = 0
    while True:
      if sum(abs(self.strat[c].iloc[zero_count]) for c in self.codes):
        break
      zero_count += 1

    sharpe  = ((return_df["daily returns"][zero_count:].sum()/100 - 
                     len(return_df[zero_count:]) * daily_rate) /
                    return_df["daily returns"][zero_count:].std())


    sortino = ((return_df["daily returns"][zero_count:].sum()/100 - 
                     len(return_df) * daily_rate) /
                    return_df["daily returns"][(return_df["daily returns"] < 0)].std())
    
    
    #plots figures if fig TRUE
    if fig_strat:
      # plot of strategy returns
      plt.figure()
      plt.title("Strategy Backtest from "+ start_date+" to "+end_date)
      plt.plot(return_df["cumulative returns"])
      plt.show()

    if fig_other:
      # plot of all individual stocks
      for c in self.codes:
        plt.figure()
        plt.title("Buy and hold from "+ start_date+" to "+end_date+" for "+str(c))
        plt.plot(((self.data["Adj Close"][c].pct_change()+1).cumprod()-1)*100)
        plt.show()
    
    return [return_df, sharpe, sortino]


## Strategy Specific Classes
Each strategy will have to have at a minimum a specific backtesting method defined in the subclass. This method is where the trading algorithm will be implemented and will return a pandas dataframe with an index of all trading dates, columns for each stock in the backtest and data containing values between -1 and 1 representing the positon in each asset at each date.

## Buy and Hold Implementation
The first strategy example that we will brifly look at is simply buying and holding each selected asset with an equal weighting. This is acheived by first defining a pandas dataframe full of ones and then dividing through by the number of tickers (or codes). Although this isn't much of an algorithm, it is useful for seeing how the skeleton code works. 

In [ ]:
class StrategyBuyAndHold(Strategies):
    """
    This strategy as the name suggests buys and holds an equal amount of each security
    """
    def backtest(self, start_date, end_date):
      Strategies.backtest(self, start_date, end_date)
      # creates a normalized set of weightings representing a buy and hold strat
      #with each column summing to one 
      self.strat = pd.DataFrame(data = (np.ones([len(self.data), len(self.codes)])
                    /len(self.codes)), columns = self.codes)
      return self.strat
      


In [ ]:
# testing of the buy and hold strategy
testbh = StrategyBuyAndHold(["GC=F"])
testbh.evaluate("2018-01-01","2022-03-05", fig_other=True)

## Coursework Task
****
As we were unable to cover this topic in lectures the aim of this coursework will be to code up a moving average model. The concept behind this model involves comparing the short term moving average (e.g average over the last 5 days) with a long term moving average (e.g average over the last 200 days) at each time point. This comparison is what produces buy/sell signals.
****
### Strategy Outline
This strategy has 2 parameters. 
- **s** - the lookback period for calculating the moving average over a **short** time period.
- **l** - the lookback period for calculating the moving average over a **large** time period.

At each time instance it is necessary to calculate the simple moving average of the previous **s** and **l** asset prices. To do this in code you could for example do ***self.data["Adj Close"][c][i-s:i].mean()*** i,e calculate the mean of the previous s values at time step i for stock code or ticker c.

Then if the short moving average crosses the long moving average i.e the value of the **small time period MA is greater than the large time period MA** then this is a **buy** signal. Otherwise enter a neutral position. (for this strategy there will be **no short selling**) 

For an individual security at a ceratin time point a buy signal corresponds to a positive value in self.strat and a neutral postion corresponds to a 0. Remember to also normalize across each row of self.strat as was done in the time series momentum strategy in lecture 1.
****
### Extension
An optional part of the coursework will be to extend the complexity of the strategy. You are free to investigate or develop any extra code that you like although I will include a few examples of how to extend the above model:
- Instead of using simple moving average's try to code **exponential moving averages** or attempt to use **Kauffmann's adoptive moving average**. For more info on this check out https://corporatefinanceinstitute.com/resources/knowledge/trading-investing/kaufmans-adaptive-moving-average-kama/
- Implement a **weighting scheme**.
- Investigate how **varying the parameters** of the number of days used in calculating moving averages effects returns and/or volatilty. Try discussing how you would tune the parameters in the question.

If you do decide to complete an extension please include a brief summary of what it is you aim to implement in a text block and try to make your code readable by including comments etc... .

**For those attempting an extension make sure all the code is error free or else we will not be able to mark your extra code.**
****
### Marking
The coursework will be marked out of 30 with 20 marks for completing the basic strategy and the remaining 10 marks for extending on the strategy as discussed above.

The 20 marks for the basic strategy will be awarded for the following:
- Getting something that works as described above **(10 marks)**

- Usage of **Pandas and Numpy** to optimise how the code runs. For example there are lots of useful inbuilt pandas and numpy functions that can make your code considerably shorter and easier to follow including ***pd.rolling(), pd.pct_change()*** to name a couple. The following 2 links are for the pandas and numpy documentation repsectively, https://pandas.pydata.org/docs/ and https://numpy.org/doc/ **(5 marks)**

- Readability of the code and efficiency. Try to include a few comments although don't go overboard. **(3 marks)**

- A brief summary of how the code works. This can be written in a text cell below below where you have written your strategy code. **(2 marks)**
****
**On completion of the coursework and accompanying quiz candiates will recieve a grade of either a pass, merit or distinction.**

**To submit your coursework you should upload your final work to the microsoft form given in the email**

In [ ]:
class StrategySMA(Strategies):
  """
  An implementation of the simple moving average crossover strategy
  """
  def backtest(self, start_date, end_date, s, l):
    """
    s is the length of the shorter time period,
    l is the length of the longer time period.
    """
    Strategies.backtest(self, start_date, end_date)
    ##### ENTER YOUR CODE HERE #####

In [ ]:
# testing of the moving average crossover strategy
testsma = StrategySMA(["GC=F","ZC=F","HG=F"])
testsma.evaluate("2018-01-01","2022-03-05", fig_other=True)

To implement the extension method discussed above feel free to add any extra code and text blocks to help with your strategy and analysis.
